In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
import pickle
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', 100)

# Scrape data

In [3]:
def getAndParseUrl(url):
    results = requests.get(url)
    time.sleep(1)
    soup = BeautifulSoup(results.text, 'html.parser')
    return soup

In [27]:
def get_players(): # function to grab every player and his stats from the page as a dictionary, and store as list
    
    url1 = 'https://www.basketball-reference.com/play-index/psl_finder.cgi?request=1&match=single&type=totals&per_minute_base=36&per_poss_base=100&season_start=1&season_end=-1&lg_id=NBA&age_min=0&age_max=99&is_playoffs=N&height_min=0&height_max=99&year_min=1980&year_max=2019&birth_country_is=Y&as_comp=gt&as_val=0&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&c1stat=per&c1comp=gt&c1val=-100&c2stat=gs&c2comp=gt&c2val=-1&c3stat=ws&c3comp=gt&c3val=-100&order_by=season&order_by=&offset='
    
    # player dictionaries container
    players = []
    
    # loop through each page(for this website, each page is accessed by 'offset={some multiple of 100}')
    # last page is offset=16000
    for i in tqdm(range(0,16100,100)):
        
        # add the offset number to the end of the url to go through each page
        url = url1+str(i)
        
        # parse each page for html
        soup = getAndParseUrl(url)
        
        # select each td tag under tr tag: each td tag in player_soup is a player's stat, all stored as a nested list
        # each element in this list is a list of the players
        player_soup = [t.select('td') for t in soup.select('#stats tbody tr') if 'class' not in t.attrs.keys()]
        
        # loop through each stat list in the player_soup
        for stat_list in tqdm(player_soup):
            
            # each player container (player with stats)
            each_player = {}
            
            #loop through p to create dictionary of each player
            for stat in stat_list:
                # set each stat equal to the tag text (i.e. pts_per_g = 15.4)
                each_player[stat.attrs['data-stat']] = stat.text
            
            # add each player dictionary to the players container
            players.append(each_player)
            
        # check to see if players list is actually getting filled
        print(len(players))    
        
        # check current page to see if there is a next page
        # if not, current page is last page, and break out of looping through offset pages
        next_pg_checker = soup.select('p a[href]')
        if 'Next page' not in [c.text for c in next_pg_checker]:
            break
        else: continue
        
    return players

In [28]:
players=get_players()





  0%|          | 0/161 [00:00<?, ?it/s]




  0%|          | 0/100 [00:00<?, ?it/s]




100%|██████████| 100/100 [00:00<00:00, 7439.88it/s]



  1%|          | 1/161 [00:06<17:33,  6.59s/it]

100







  0%|          | 0/100 [00:00<?, ?it/s]




100%|██████████| 100/100 [00:00<00:00, 2451.10it/s]



  1%|          | 2/161 [00:11<16:31,  6.23s/it]

200







  0%|          | 0/100 [00:00<?, ?it/s]




100%|██████████| 100/100 [00:00<00:00, 4792.94it/s]

...

15900







  0%|          | 0/100 [00:00<?, ?it/s]




100%|██████████| 100/100 [00:00<00:00, 8300.95it/s]



 99%|█████████▉| 160/161 [14:15<00:05,  5.47s/it]

16000







  0%|          | 0/44 [00:00<?, ?it/s]




100%|██████████| 44/44 [00:00<00:00, 5334.11it/s]

16044


In [29]:
# check to see if all player dicts were created
len(players)

# THEY WERE

16044

In [30]:
# save the players as a pickle
with open('player_season_totals.pickle', 'wb') as outfile:
    pickle.dump(players, outfile)

outfile.close()

In [31]:
players

[{'player': 'Álex Abrines',
  'age': '25',
  'team_id': 'OKC',
  'lg_id': 'NBA',
  'per': '6.3',
  'gs': '2',
  'ws': '0.6',
  'season': '2018-19',
  'g': '31',
  'mp': '588',
  'fg': '56',
  'fga': '157',
  'fg2': '15',
  'fg2a': '30',
  'fg3': '41',
  'fg3a': '127',
  'ft': '12',
  'fta': '13',
  'orb': '5',
  'drb': '43',
  'trb': '48',
  'ast': '20',
  'stl': '17',
  'blk': '6',
  'tov': '14',
  'pf': '53',
  'pts': '165',
  'fg_pct': '.357',
  'fg2_pct': '.500',
  'fg3_pct': '.323',
  'efg_pct': '.487',
  'ft_pct': '.923',
  'ts_pct': '.507'},
 {'player': 'Quincy Acy',
  'age': '28',
  'team_id': 'PHO',
  'lg_id': 'NBA',
  'per': '2.9',
  'gs': '0',
  'ws': '-0.1',
  'season': '2018-19',
  'g': '10',
  'mp': '123',
  'fg': '4',
  'fga': '18',
  'fg2': '2',
  'fg2a': '3',
  'fg3': '2',
  'fg3a': '15',
  'ft': '7',
  'fta': '10',
  'orb': '3',
  'drb': '22',
  'trb': '25',
  'ast': '8',
  'stl': '1',
  'blk': '4',
  'tov': '4',
  'pf': '24',
  'pts': '17',
  'fg_pct': '.222',
  'fg2